In [2]:
import pandas as pd
import plotly.express as px
from src.pipelines.data_quality import insight_dirty_cost
df_dirty, df_clean = insight_dirty_cost()

In [3]:
# df_clean
df1 = df_clean.groupby('id_store').agg({
    'id_order': ['count'],
    'price': ['sum']
})
df1 = df1.reset_index()
df1.columns = ['id_store', 'count_orders', 'sum_orders']
df1.sort_values(['sum_orders', 'count_orders'], ascending=False)

,id_store,count_orders,sum_orders
8,LJ-09,420,159809.03
6,LJ-07,433,156908.79
2,LJ-03,411,154381.59
9,LJ-10,401,154293.39
3,LJ-04,403,150381.52
0,LJ-01,393,148146.81
7,LJ-08,409,147956.84
1,LJ-02,406,146578.48
5,LJ-06,406,143080.62
4,LJ-05,373,130648.08


In [4]:
df2 = df_dirty.groupby('loja_id').agg({
    'id_venda': ['count'],
    'valor': ['sum']
})

df2 = df2.reset_index()
df2.columns = ['id_store_dirty', 'count_orders_dirty', 'sum_orders_dirty']
df2.sort_values('sum_orders_dirty', ascending=False)

,id_store_dirty,count_orders_dirty,sum_orders_dirty
8,LJ-09,468,176925.66
6,LJ-07,477,176161.23
2,LJ-03,471,175553.97
9,LJ-10,447,172019.75
0,LJ-01,448,169830.84
3,LJ-04,453,168096.71
7,LJ-08,456,165283.25
1,LJ-02,450,163665.91
5,LJ-06,452,158333.72
4,LJ-05,423,147915.43


In [5]:
df_merge = pd.merge(df2, df1, left_on='id_store_dirty', right_on='id_store', how='left')
df_merge

,id_store_dirty,count_orders_dirty,sum_orders_dirty,id_store,count_orders,sum_orders
0,LJ-01,448,169830.84,LJ-01,393,148146.81
1,LJ-02,450,163665.91,LJ-02,406,146578.48
2,LJ-03,471,175553.97,LJ-03,411,154381.59
3,LJ-04,453,168096.71,LJ-04,403,150381.52
4,LJ-05,423,147915.43,LJ-05,373,130648.08
5,LJ-06,452,158333.72,LJ-06,406,143080.62
6,LJ-07,477,176161.23,LJ-07,433,156908.79
7,LJ-08,456,165283.25,LJ-08,409,147956.84
8,LJ-09,468,176925.66,LJ-09,420,159809.03
9,LJ-10,447,172019.75,LJ-10,401,154293.39


In [104]:
fig = px.bar(df_merge, 
             x='id_store_dirty', 
             y=['sum_orders_dirty', 'sum_orders'],
             labels={'id_store_dirty': 'Código de Loja', 'value': 'Total Faturamento Loja'},
             title='Custo da Sujeira: Faturamento Antes x Depois da Limpeza de cada Loja',
             barmode='group',
             color_discrete_map={'sum_orders_dirty': 'red', 'sum_orders': 'blue'})

fig.update_traces(selector={'name': 'sum_orders_dirty'}, name='Faturamento Pré-Limpeza')
fig.update_traces(selector={'name': 'sum_orders'}, name='Faturamento Pós-Limpeza')
fig.update_layout(legend=dict(title='Tipos de Faturameto', font=dict(size=12, color='black')), title_x=0.1)       
fig.show()

In [90]:
df_sum = df_merge[['count_orders_dirty', 'count_orders']].sum()
df_sum['diff'] = df_sum['count_orders_dirty'] - df_sum['count_orders']
df_pie = df_sum[['count_orders_dirty', 'diff']]
df_pie = df_pie.reset_index()
df_pie.columns = ['labels', 'values']
df_pie

,labels,values
0,count_orders_dirty,4545
1,diff,490


In [103]:
fig = px.pie(df_pie,
             names='labels',
             values='values',
             title='Proporção Pedidos Perdidos',
             hole=0.5)

fig.update_traces(labels=['Pedidos Pré-Limpeza', 'Pedidos Perdidos Pós-Limpeza'])
fig.update_traces(textinfo='percent+value')
fig.update_layout(legend=dict(title='Tipos de Pedidos', font=dict(size=12, color='black')), title_x=0.1) 
fig.show()